In [ ]:
import gradio as gr
import requests
from langchain.llms import Ollama

# TMDB API 키
TMDB_KEY = "97f5ea5f12fa2ebcb7d31beb4ad328b0"

# Ollama 모델 사용할 때는 API 키 필요 없습니다
# Ollama 모델 초기화
model = Ollama(model="gemma2", base_url="http://localhost:11434")

# 1. TMDB API 요청 함수
# "TMDB 정보"를 받아옴
def flim(title):
    url = "https://api.themoviedb.org/3/search/movie"
    # 주소 뒤 파라미터
    params = {
        'api_key': TMDB_KEY,
        'query': title,
        'language': 'ko-KR'
    }

    try:
        #url 응답받기
        response = requests.get(url, params=params)
        
        # API 응답 상태 확인
        response.raise_for_status()  # HTTP 에러 발생 시 예외 발생
        
        # "json" data로 응답
        data = response.json()
                       # 에러를 방지해줍니다.
        results = data.get('results', [])
        print("결과:", results)
        print("결과2:", results[0])
        # 결과가 없는 경우 처리
        if not results:
            return f"'{title}'에 대한 영화를 찾을 수 없습니다. 다른 제목을 입력해 보세요."

        # 첫 번째 영화 "정보 추출"
        # "첫번째"만 가져옴 [0]
        movie = results[0]
        movie_info = f"""🎬 제목: {movie.get('title')}
        📅 개봉일: {movie.get('release_date')}
        ⭐ 평점: {movie.get('vote_average')}/10
        📖 줄거리: {movie.get('overview')}
        """

        # 포스터만 보여주면 됨
        poster_path = movie.get('poster_path')
        poster_url = f"https://image.tmdb.org/t/p/w500{poster_path}"
        # "https://image.tmdb.org/t/p/w500{poster_path}" : 포스터 경로 
        return movie_info, poster_url

    except requests.exceptions.RequestException as e:
        return f"API 요청 중 오류 발생: {str(e)}"
    except IndexError:
        return f"영화 정보를 처리하는 중 오류 발생: 검색 결과가 없습니다."

# 2. 챗봇 함수
def flim_chatbot(message):
    # 1. TMDB로부터 영화 정보 받아오기
    movie_info = flim(message)
    
    # 영화 정보를 찾지 못한 경우 그대로 반환
    if "찾을 수 없습니다" in movie_info or "오류 발생" in movie_info:
        return movie_info

    # 튜플 언패킹(Tuple Unpacking)
    # 분리
    movie_info, poster_url = movie_info

    # 2. AI 모델에 전달할 "프롬프트 구성과 이해"
    prompt = f"""당신은 친근하고 전문적인 영화 전문가 챗봇입니다. 
다음 영화 정보를 바탕으로 사용자에게 재미있고 유익한 정보를 제공해주세요.

영화 정보: {movie_info} 

사용자 질문: {message}

사용자들이 ~설명해주세요 혹은 ~알려주세요라고 해도 영화정보를 알려주시면 됩니다.
위 정보를 바탕으로 영화에 대해 친근하고 상세하게 설명해주세요."""

    try:
        # Ollama 모델 호출 
        # Ollama 모델에 전달하여 응답을 생성합니다. (invoke)
        response = model.invoke(prompt)
        return poster_url, response
    
    except Exception as e:
        return f"챗봇 응답 생성 중 오류 발생: {str(e)}"

# Gradio 인터페이스 생성
iface = gr.Interface(
    # 함수만 받습니다
    fn=flim_chatbot,
    # example
    inputs=gr.Textbox(lines=10, placeholder="영화 제목을 입력하세요 (예: 기생충)"),
    outputs=[gr.Image(label="포스터", show_label=True),  
             gr.Textbox(label="상세설명", lines=20)
                   
    ],
    title="🎬 영화 정보 챗봇",
    description="영화 제목을 입력하면 상세한 정보와 함께 AI가 영화에 대해 설명해드립니다!",
    # examples=[
    #     ["기생충"],
    #     ["어벤져스가 뭐야?"],
    #     ["타이타닉에 대해 알려줘"],
    #     ["인터스텔라는 재미있어?"],
    #     ["기생충이라는 영화 있어?"],
    #     ["탑건 설명해줘"]
    # ]
)




# TMDB API 요청 함수
def TV(title):
    url = "https://api.themoviedb.org/3/search/tv"
    # 주소 뒤 파라미터
    params = {
        'api_key': TMDB_KEY,
        'query': title,
        'language': 'ko-KR'
    }

    try:
        #url 응답받기
        response = requests.get(url, params=params)
        
        # API 응답 상태 확인
        response.raise_for_status()  # HTTP 에러 발생 시 예외 발생
        
        # json data로 응답
        data = response.json()
                       # 에러를 방지해줍니다.
        results = data.get('results', [])
        print("결과:", results)
        print("결과2:", results[0])
        # 결과가 없는 경우 처리
        if not results:
            return f"'{title}'에 대한 프로그램을 찾을 수 없습니다. 다른 제목을 입력해 보세요."

        # 첫 번째 영화 "정보 추출"
        # "첫번째"만 가져옴 [0]
        show = results[0]
        show_info = f"""🎬 제목: {show.get('title')}
        📅 개봉일: {show.get('release_date')}
        ⭐ 평점: {show.get('vote_average')}/10
        📖 줄거리: {show.get('overview')}
        """
        
        
        # 포스터만 보여주면 됨
        poster_path = show.get('poster_path')
        poster_url = f"https://image.tmdb.org/t/p/w500{poster_path}"
        # "https://image.tmdb.org/t/p/w500{poster_path}" : 포스터 경로 
        return show_info, poster_url

    except requests.exceptions.RequestException as e:
        return f"API 요청 중 오류 발생: {str(e)}"
    except IndexError:
        return f"영화 정보를 처리하는 중 오류 발생: 검색 결과가 없습니다."

# 챗봇 함수
def show_chatbot(message):
    # 1. TMDB로부터 드라마 정보 받아오기
    show_info = TV(message)
    
    # 드라마 정보를 찾지 못한 경우 그대로 반환
    if "찾을 수 없습니다" in show_info or "오류 발생" in show_info:
        return show_info

    # 튜플 언패킹(Tuple Unpacking)
    # 분리
    show_info, poster_url = show_info

    # 2. AI 모델에 전달할 "프롬프트 구성과 이해"
    prompt = f"""당신은 친근하고 전문적인 TV프로그램 전문가 챗봇입니다. 
다음 TV프로그램 정보를 바탕으로 사용자에게 재미있고 유익한 정보를 제공해주세요.


TV프로그램 정보: {show_info} 

사용자 질문: {message}

사용자들이 ~설명해주세요 혹은 ~알려주세요라고 해도 영화정보를 알려주시면 됩니다.
위 정보를 바탕으로 TV프로그램에 대해 친근하고 상세하게 설명해주세요."""

    try:
        # Ollama 모델 호출 
        # Ollama 모델에 전달하여 응답을 생성합니다. (invoke)
        response = model.invoke(prompt)
        return poster_url, response
    
    except Exception as e:
        return f"챗봇 응답 생성 중 오류 발생: {str(e)}"

# Gradio 인터페이스 생성
iface2 = gr.Interface(
    # 함수만 받습니다
    fn=show_chatbot,
    # example
    inputs=gr.Textbox(lines=10, placeholder="TV프로그램 제목을 입력하세요 (예: 고든램지의 헬스키친)"),
    outputs=[gr.Image(label="포스터", show_label=True),  
             gr.Textbox(label="상세설명", lines=20)
                   
    ],
    title="🎬 TV프로그램 정보 챗봇",
    description="TV프로그램 제목을 입력하면 상세한 정보와 함께 AI가 TV프로그램에 대해 설명해드립니다!",
    # examples=[
    #     ["기생충"],
    #     ["어벤져스가 뭐야?"],
    #     ["타이타닉에 대해 알려줘"],
    #     ["인터스텔라는 재미있어?"],
    #     ["기생충이라는 영화 있어?"],
    #     ["탑건 설명해줘"]
    # ]
)


app = gr.TabbedInterface([iface, iface2], ["영화", "드라마"])


# 서버 실행
# if __name__ == "__main__":
app.launch(server_port=7875, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7875
* To create a public link, set `share=True` in `launch()`.


결과: [{'adult': False, 'backdrop_path': '/mLyW3UTgi2lsMdtueYODcfAB9Ku.jpg', 'genre_ids': [35], 'id': 2316, 'origin_country': ['US'], 'original_language': 'en', 'original_name': 'The Office', 'overview': '팬실베니아 스크랜튼에 위치한 사무용지 판매회사 던더 미플린의 스크랜턴 지점. 무미건조한 사무실에서 깐깐한 지점장과 독특한 캐릭터의 부하들이 벌이는 코믹 해프닝과 가슴 찡한 이야기가 펼쳐진다. BBC에서 방영된 동명 영국 시리즈의 할리우드 리메이크.', 'popularity': 132.4912, 'poster_path': '/7DJKHzAi83BmQrWLrYYOqcoKfhR.jpg', 'first_air_date': '2005-03-24', 'name': '오피스', 'vote_average': 8.58, 'vote_count': 4531}, {'adult': False, 'backdrop_path': '/7asHgfnYs757TSWftNwcQHI59BS.jpg', 'genre_ids': [35, 18], 'id': 9279, 'origin_country': ['JP'], 'original_language': 'ja', 'original_name': 'ショムニ', 'overview': 'Shomuni, that\'s the common name given to General Affairs Section 2 of a trading company. Here are six women who have ended up in this department known as "the dump" for office ladies. But Chinatsu Tsuboi (Makiko Esumi) declares that the value of woman is equal to the number of men. With her as

In [ ]:
iface.close()